In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Thyroid_Cancer/GSE151181'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene and miRNA expression in radioiodine refractory and avid papillary thyroid carcinomas"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['histological variant: Classical', 'histological variant: Follicular', 'histological variant: NA', 'histological variant: non-neoplastic thyroid'], 1: ['tissue type: Primary tumor', 'tissue type: synchronous lymph node metastasis', 'tissue type: lymph node metastasis post RAI', 'tissue type: lymph node metastasis_2 post RAI', 'tissue type: lymph node metastasis_1 post RAI', 'tissue type: non-neoplastic thyroid'], 2: ['collection before/after rai: Before', 'collection before/after rai: After'], 3: ['patient id: pt_1', 'patient id: pt_2', 'patient id: pt_3', 'patient id: pt_5', 'patient id: pt_7', 'patient id: pt_8', 'patient id: pt_11', 'patient id: pt_12', 'patient id: pt_13', 'patient id: p

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = True
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

import re

# Analyzing dataset availability
# Based on the output from STEP 1, this dataset seems to contain gene expression data.

# Variable Availability

# Thyroid_Cancer (Trait) -- This is inferred as the histological variant.
trait_row = 0 

# Age and Gender seem to be missing from given characteristics data.
# Analyzing sample characteristics dictionary, age and gender data are not found explicitly.

# Data Type Conversion Functions

def extract_value(text):
    """ Extract the value from the text after colon """
    if ':' in text:
        return text.split(':', 1)[1].strip()
    return None

def convert_trait(value):
    """ Convert trait to binary, 1 if pathological variant else 0 """
    value_mapped = {
        'Classical': 1,
        'Follicular': 1,
        'non-neoplastic thyroid': 0,
        'NA': None
    }
    extracted_value = extract_value(value)
    return value_mapped.get(extracted_value, None)

# Age and Gender are not available.
convert_age = lambda value: None
convert_gender = lambda value: None

# Saving cohort information
save_cohort_info('GSE151181', './preprocessed/Thyroid_Cancer/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction
# Clinical data is available since trait_row is not None
selected_clinical_data = geo_select_clinical_features(clinical_data, 'Thyroid_Cancer', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
csv_path = './preprocessed/Thyroid_Cancer/trait_data/GSE151181.csv'
selected_clinical_data.to_csv(csv_path)
print(preview_df(selected_clinical_data))


{'GSM4567964': [1], 'GSM4567965': [1], 'GSM4567966': [1], 'GSM4567967': [1], 'GSM4567968': [1], 'GSM4567969': [1], 'GSM4567970': [1], 'GSM4567971': [1], 'GSM4567972': [1], 'GSM4567973': [1], 'GSM4567974': [1], 'GSM4567975': [1], 'GSM4567976': [1], 'GSM4567977': [1], 'GSM4567978': [1], 'GSM4567979': [1], 'GSM4567980': [1], 'GSM4567981': [1], 'GSM4567982': [1], 'GSM4567983': [1], 'GSM4567984': [1], 'GSM4567985': [1], 'GSM4567986': [1], 'GSM4567987': [1], 'GSM4567988': [1], 'GSM4567989': [1], 'GSM4567990': [1], 'GSM4567991': [1], 'GSM4567992': [1], 'GSM4567993': [1], 'GSM4567994': [1], 'GSM4567995': [1], 'GSM4567996': [1], 'GSM4567997': [1], 'GSM4567998': [1], 'GSM4567999': [None], 'GSM4568000': [0], 'GSM4568001': [0], 'GSM4568002': [0], 'GSM4568003': [0], 'GSM4568004': [0], 'GSM4568005': [0], 'GSM4568006': [0], 'GSM4568007': [0], 'GSM4568008': [0], 'GSM4568009': [0], 'GSM4568010': [0]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '13', '14',
       '15', '16', '17', '18', '21', '23', '24'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1', '2', '3', '4', '5'], 'COL': ['192', '192', '192', '192', '192'], 'ROW': ['328', '326', '324', '322', '320'], 'SPOT_ID': ['miRNABrightCorner30', 'Blank', 'Blank', 'Blank', 'Blank'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'pos', 'pos'], 'miRNA_ID': [nan, nan, nan, nan, nan], 'GENE_SYMBOL': [nan, nan, nan, nan, nan], 'GENE_NAME': [nan, nan, nan, nan, nan], 'ACCESSION_STRING': [nan, nan, nan, nan, nan], 'CHROMOSOMAL_LOCATION': [nan, nan, nan, nan, nan]}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Set the identifier and gene symbol keys
identifier_key = 'ID'
gene_symbol_key = 'GENE_SYMBOL'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to get the gene expression dataframe
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Thyroid_Cancer/gene_data/GSE151181.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Thyroid_Cancer')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE151181', './preprocessed/Thyroid_Cancer/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Thyroid_Cancer/GSE151181.csv'
    unbiased_merged_data.to_csv(csv_path)


No gene data in the dataframe
